In [1]:
!gsutil -m cp -R gs://natural_questions/v1.0 .

Copying gs://natural_questions/v1.0/dev/nq-dev-03.jsonl.gz...
Copying gs://natural_questions/v1.0/dev/nq-dev-00.jsonl.gz...
Copying gs://natural_questions/v1.0/sample/nq-train-sample.jsonl.gz...
Copying gs://natural_questions/v1.0/README.txt...
Copying gs://natural_questions/v1.0/dev/nq-dev-04.jsonl.gz...
Copying gs://natural_questions/v1.0/LICENSE.txt...
Copying gs://natural_questions/v1.0/sample/nq-dev-sample.jsonl.gz...
Copying gs://natural_questions/v1.0/dev/nq-dev-01.jsonl.gz...
Copying gs://natural_questions/v1.0/dev/nq-dev-02.jsonl.gz...
Copying gs://natural_questions/v1.0/train/nq-train-00.jsonl.gz...
Copying gs://natural_questions/v1.0/train/nq-train-01.jsonl.gz...
Copying gs://natural_questions/v1.0/train/nq-train-02.jsonl.gz...
Copying gs://natural_questions/v1.0/train/nq-train-03.jsonl.gz...
Copying gs://natural_questions/v1.0/train/nq-train-04.jsonl.gz...
Copying gs://natural_questions/v1.0/train/nq-train-05.jsonl.gz...
Copying gs://natural_questions/v1.0/train/nq-train-06

In [2]:
import gzip
import json
import os
from collections import defaultdict
from html.parser import HTMLParser

import numpy as np
import pandas as pd
import six
import tqdm.notebook as tq

In [3]:
cd v1.0/

/content/v1.0


In [4]:
mkdir stats

In [5]:
file_name = "nq-train-00.jsonl.gz"
print(file_name.split(".")[0])
data = defaultdict(list)
file_path = os.path.join("train", file_name)
with gzip.GzipFile(file_path, "rb") as fi:
    for compt, line in tq.tqdm(enumerate(fi)):
        if not isinstance(line, six.text_type):
            line = line.decode("utf-8")
        json_example = json.loads(line)
        doc_html = json_example["document_html"]
        break

nq-train-00


In [6]:
from lxml import etree

In [7]:
root = etree.HTML(doc_html)

In [18]:
root = etree.HTML(doc_html)
find = etree.XPath("//body")
new_etree = find(root)[0]
sub_html = etree.tostring(new_etree, encoding="UTF-8", method="html").decode("UTF-8")
print(sub_html)

<body class="mediawiki ltr sitedir-ltr mw-hide-empty-elt ns-0 ns-subject page-The_Walking_Dead_season_8 rootpage-The_Walking_Dead_season_8 skin-vector action-view">		<div id="mw-page-base" class="noprint"></div>
		<div id="mw-head-base" class="noprint"></div>
		<div id="content" class="mw-body" role="main">
			<a id="top"></a>
			<div id="siteNotice" class="mw-body-content"><div id="centralNotice"></div><!-- CentralNotice --></div><div class="mw-indicators mw-body-content">
</div>
<h1 id="firstHeading" class="firstHeading" lang="en"><i>The Walking Dead</i> (season 8)</h1>			<div id="bodyContent" class="mw-body-content">
				<div id="siteSub" class="noprint">From Wikipedia, the free encyclopedia</div>				<div id="contentSub"></div>
								<div id="jump-to-nav" class="mw-jump">
					Jump to:					<a href="#mw-head">navigation</a>, 					<a href="#p-search">search</a>
				</div>
				<div id="mw-content-text" lang="en" dir="ltr" class="mw-content-ltr"><div class="mw-parser-output"><table cl

# Tests of tokenization

In [36]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 3.1 MB/s 
     |████████████████████████████████| 636 kB 29.4 MB/s 
     |████████████████████████████████| 3.3 MB 37.2 MB/s 
     |████████████████████████████████| 895 kB 66.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from tokenizers import Regex, normalizers
from transformers import AutoTokenizer

In [ ]:
from html.entities import name2codepoint
from html.parser import HTMLParser


class MyHTMLParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.text = None

    def handle_starttag(self, tag, attrs):
        if tag == "body":
            self.text = ""

    def handle_data(self, data):
        # print("Data     :", data)
        if self.text is not None:
            self.text += f" {data}"

In [49]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")

In [68]:
normalizer = normalizers.Sequence(
    [
        normalizers.Nmt(),
        normalizers.Replace(Regex(" {2,}"), " "),
    ]
)

In [71]:
file_name = "nq-train-00.jsonl.gz"
print(file_name.split(".")[0])
data = defaultdict(list)
file_path = os.path.join("train", file_name)
tokenized_lengths = []
tokenized_lengths_reduced = []
with gzip.GzipFile(file_path, "rb") as fi:
    for compt, line in tq.tqdm(enumerate(fi)):
        if not isinstance(line, six.text_type):
            line = line.decode("utf-8")
        json_example = json.loads(line)
        doc_html = json_example["document_html"]
        parser = MyHTMLParser()
        parser.feed(doc_html)
        out = tokenizer.encode(parser.text)
        tokenized_lengths.append(len(out))
        reduce_text = normalizer.normalize_str(parser.text)
        out = tokenizer.encode(reduce_text)
        tokenized_lengths_reduced.append(len(out))

nq-train-00


In [76]:
def print_stats(tokenized_lengths):
    tokenized_lengths = np.array(tokenized_lengths)
    print("mean: ", tokenized_lengths.mean())
    print(
        "number of example below model_max_length: ",
        (tokenized_lengths < tokenizer.model_max_length).sum(),
    )
    print("min tokenized len: ", tokenized_lengths.min())


print("Raw text (without HTML tags)")
print_stats(tokenized_lengths)
print("\nRaw text (without HTML tags) reduced with NMT + deduplicate space:")
print_stats(tokenized_lengths_reduced)

Raw text (without HTML tags)
mean:  11348.072974333165
number of example below model_max_length:  0
min tokenized len:  1285

Raw text (without HTML tags) reduced with NMT + deduplicate space:
mean:  9215.2653917128
number of example below model_max_length:  168
min tokenized len:  332


# Parse dataset

In [131]:
import pprint
import re
from dataclasses import dataclass
from html.entities import name2codepoint
from html.parser import HTMLParser
from typing import List, Optional, Tuple

In [132]:
@dataclass
class HtmlTag:
    tag: str
    attrs: List[Optional[Tuple[str]]]


@dataclass
class TagToRemoveWithContent:
    tag: str
    content_min_char_length: int = -float("inf")
    content_max_char_length: int = float("inf")


@dataclass
class Metadata:
    char_start_idx: int
    value: HtmlTag
    char_end_idx: Optional[int] = None
    key: str = "html"
    type: str = "local"

In [145]:
class AttributeCleaner:
    def __init__(self, attrs_to_keep: Optional[List[str]]):
        self.attrs_to_keep = attrs_to_keep if isinstance(attrs_to_keep, list) else []

    def __call__(self, attrs: List[Tuple[str]]):
        if not isinstance(attrs, list):
            raise TypeError(f"attrs need to be a list not a {type(attrs)}")
        return [(attr, value) for attr, value in attrs if attr in self.attrs_to_keep]


class TagFilter:
    def __init__(
        self,
        tags_to_remove_alone: Optional[List[str]],
        tags_to_remove_with_content: Optional[List[TagToRemoveWithContent]],
    ):
        self.tags_to_remove_alone = (
            tags_to_remove_alone if isinstance(tags_to_remove_alone, list) else []
        )
        self.tags_to_remove_with_content = (
            {
                tag_to_remove.tag: tag_to_remove
                for tag_to_remove in tags_to_remove_with_content
            }
            if isinstance(tags_to_remove_with_content, list)
            else {}
        )

        # todo sanitize tags_to_remove_with_content

    def handle_starttag(self, tag: str):
        if not isinstance(tag, str):
            raise TypeError(f"tag need to be a string not a {type(attrs)}")
        return True if tag not in self.tags_to_remove_alone else None

    def handle_endtag(self, metadata: Metadata, current_chr_idx: int):
        if metadata.value.tag not in self.tags_to_remove_with_content:
            return True

        tag_to_remove_characteristics = self.tags_to_remove_with_content[
            metadata.value.tag
        ]
        content_char_length = current_chr_idx - metadata.char_start_idx
        if (
            char_start_idx <= tag_to_remove_characteristics.content_max_char_length
            and tag_to_remove_characteristics.content_min_char_length <= char_start_idx
        ):
            return True

        return False


class ConsecutiveTagCleaner:
    pass


class MyHTMLParser(HTMLParser):
    def __init__(
        self,
        text_separator: str = " ",
        tags_to_remove_with_content: Optional[List[TagToRemoveWithContent]] = None,
        tags_to_remove_alone: Optional[List[str]] = None,
        attrs_to_keep: Optional[List[str]] = None,
        first_tag: Optional[str] = "body",
    ):
        HTMLParser.__init__(self)

        # Utils
        self._metadata_stack = []
        self._attribute_cleaner = AttributeCleaner(attrs_to_keep=attrs_to_keep)
        self._tag_filter = TagFilter(
            tags_to_remove_alone=tags_to_remove_alone,
            tags_to_remove_with_content=tags_to_remove_with_content,
        )
        self._first_tag = first_tag
        self._num_tags_type_first_tag = 0  # assume 1st tag is a closing type
        self._collect = False if self._first_tag is not None else True
        self._text_separator = text_separator
        self._last_char_idx = 0

        # Final data collectors
        self.text = ""
        self.metadata = []

    def handle_starttag(self, tag, attrs):
        print("starttag: ", tag)
        if tag == self._first_tag:
            self._num_tags_type_first_tag += 1
            self._collect = True

        if not self._collect:
            return

        if not self._tag_filter.handle_starttag(tag):
            return

        self._metadata_stack.append(
            Metadata(
                char_start_idx=self._last_char_idx,
                value=HtmlTag(tag=tag, attrs=self._attribute_cleaner(attrs)),
            )
        )

    def handle_endtag(self, tag):
        if not self._collect:
            return

        if not self._metadata_stack:
            return

        metadata_poped = self._metadata_stack.pop()

        while metadata_poped.value.tag != tag:
            if self._tag_filter.handle_endtag(
                metadata_poped, current_chr_idx=self._last_char_idx
            ):
                self.metadata.append(metadata_poped)
            metadata_poped = self._metadata_stack.pop()

        if self._tag_filter.handle_endtag(
            metadata_poped, current_chr_idx=self._last_char_idx
        ):
            metadata_poped.char_end_idx = self._last_char_idx
            self.metadata.append(metadata_poped)

        if tag == self._first_tag:
            self._num_tags_type_first_tag -= 1

        if self._num_tags_type_first_tag < 1:
            self._collect = False

    def handle_data(self, data):
        if not self._collect:
            return

        self.text += f"{self._text_separator}{data}"
        self._last_char_idx = len(self.text)

In [150]:
html_test = """
<html>
  <head>
    <title>Href Attribute Example</title>
  </head>
  <body>
    <h1>Href Attribute Example</h1>
    <p>
      <a href="https://www.freecodecamp.org/contribute/">The freeCodeCamp Contribution Page</a> shows you how and where you can contribute to freeCodeCamp's community and growth.<br>
    </p>
  </body>
</html>
"""

In [151]:
html_test = re.sub(">\s*<", "><", html_test)

In [152]:
parser = MyHTMLParser()

In [153]:
parser.feed(html_test)

starttag:  html
starttag:  head
starttag:  title
starttag:  body
starttag:  h1
starttag:  p
starttag:  a
starttag:  br


In [154]:
print(parser.text)

 Href Attribute Example The freeCodeCamp Contribution Page  shows you how and where you can contribute to freeCodeCamp's community and growth.


In [155]:
pprint.pprint(parser.metadata)

[Metadata(char_start_idx=0, value=HtmlTag(tag='h1', attrs=[]), char_end_idx=23, key='html', type='local'),
 Metadata(char_start_idx=23, value=HtmlTag(tag='a', attrs=[]), char_end_idx=58, key='html', type='local'),
 Metadata(char_start_idx=142, value=HtmlTag(tag='br', attrs=[]), char_end_idx=None, key='html', type='local'),
 Metadata(char_start_idx=23, value=HtmlTag(tag='p', attrs=[]), char_end_idx=142, key='html', type='local'),
 Metadata(char_start_idx=0, value=HtmlTag(tag='body', attrs=[]), char_end_idx=142, key='html', type='local')]


In [ ]:
# TODO: self closing tag